from pkgs.simplifier import simplify
from pkgs.compare import comp

In [144]:
from pkgs import txt

import numpy as np

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [145]:
file1 = open('doc1.txt','r')
file2 = open('doc2.txt','r')

a = file1.read()
b = file2.read()

In [146]:
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words("english"))
stop.remove("not")
stop.remove("no")
posIgnore = ["$","''","(",")",",","--",".",":","CD","FW","LS","NNP","NNPS","NNS","PRP","PRP$","SYM","TO","WDT","WP","WP$","WRB","``"]
exStr = "There is a setuid binary in the homedirectory that does the following: it makes a connection to localhost on the port you specify as a commandline argument. It then reads a line of text from the connection and compares it to the password in the previous level."

In [147]:
def getWnetPos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def simplify(parag = exStr):
    stns = sent_tokenize(parag)
    wrds = []
    for stn in stns:
        wrds.append(word_tokenize(stn))
    wrdsFilt = []
    for i in wrds:
        temp = []
        for j in i:
            if j.casefold() not in stop:
                temp.append(j)
        wrdsFilt.append(temp)

    tagged = []
    for words in wrdsFilt:
        tagged.append(nltk.pos_tag(words))
    wrdsLem = []
    for words in tagged:
        temp = []
        for word in words:
            if word[1] in posIgnore:
                temp.append((word[0],word[1]))    
            elif getWnetPos(word[1]) != None:
                temp.append((lemmatizer.lemmatize(word[0],pos=getWnetPos(word[1])),word[1]))
            else:
                temp.append((lemmatizer.lemmatize(word[0]),word[1]))
        wrdsLem.append(temp)
    #print(wrdsLemTag)
    '''
    for words in wrdsLemTag:
        temp = []
        for word in words:
            temp.append(word[0])
        wrdsLem.append(temp)
    '''
    
    syn = []
    '''
    for words in wrdsLem:
        temp = []
        for word in words:
            synons = []
            if word in ['.',',']:
                continue
            if((wrdsLemTag[wrdsLem.index(words)][words.index(word)])[1].startswith('N')):
                temp.append(word)
                continue
            for ss in wordnet.synsets(word):
                synons.append(ss)
            synons.sort()
            if(synons):
                wrd = synons[0]
            synons.clear()
            temp.append(wrd.name().partition('.')[0])
        syn.append(temp)
    '''
    for words in wrdsLem:
        temp = []
        for word,tag in words:
            if word in ['.',',']:
                continue
            temp.append((word,tag))
        syn.append(temp)

    return syn

In [148]:
def nMax(listIn, n):
    maxN = []
    list1 = listIn.copy()
    for i in range(0, n): 
        max1 = 0.0
        
        for j in range(len(list1)):     
            if list1[j] > max1:
                max1 = list1[j]  
        
        list1.remove(max1)
        maxN.append(max1)
    return maxN

In [149]:
from itertools import product
import nltk
from nltk.corpus import wordnet

def wordSimilarity(wrd1,wrd2):
    
    syns1 = wordnet.synsets(wrd1)
    syns2 = wordnet.synsets(wrd2)
    for sn1, sn2 in product(syns1, syns2):
        simil = wordnet.wup_similarity(sn1, sn2)
        if syns1 == syns2:
            return 1.0
        else:
            return simil #if (simil != None) else 0.5 # Due to None implying uncertainity


def comp(txt1,txt2):
    assert len(txt1) == len(txt2), "Different number of lines in samples."
    
    sim = np.zeros((len(txt1),len(txt2))) == 1
    
    #Word Comparasion and Scoring
    totalScore = []
    for sA,sB in zip(txt1,txt2):
        discard = []
        score = []
        for wA in sA:
            wrdScores = []
            if wA in discard: continue
            for wB in sB:
                if wB in discard: continue
                wrdScores.append(wordSimilarity(wA[0],wB[0])) 
                #print(wA[0],wB[0],wordSimilarity(wA[0],wB[0]))
            #print(wrdScores)
            wrdScores = [i if i != None else 0.1 for i in wrdScores] # Replace all uncertain values, ie. None, with a valid approximate value
            score.append(max(wrdScores))
            discard.append(sB[wrdScores.index(max(wrdScores))])
        totalScore.append(sum(nMax(score,min(len(sA),len(sB))))/len(nMax(score,min(len(sA),len(sB)))))
    
    return totalScore

In [150]:
#simplify(a)[0]
print(comp(simplify(a),simplify(b)))
#print(comp(simplify(),simplify()))

ValueError: list.remove(x): x not in list

2